In [1]:
import os
import tensorflow as tf
import json_lines as jl
import re
import pickle as pickle
from nltk.corpus import reuters
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords


ModuleNotFoundError: No module named 'json_lines'

In [ ]:
cachedStopWords = stopwords.words("english")
data_src_path = 'data/signalmedia-1m.jsonl'
output_filename = "data/tokens.pkl"

In [ ]:
def tokenize(text):
    try:
        min_length = 3
        #print text
        words = map(lambda word: word.lower(), word_tokenize(text))#text.decode('utf8', 'ignore')));
        #print "*** " + str(words)
        # words = [word for word in words
        #             if word not in cachedStopWords]
        '''tokens = (list(map(lambda token: PorterStemmer().stem(token),
                        words)));'''
        p = re.compile('[a-zA-Z]+');
        filtered_tokens = list(filter(lambda token:
                            p.match(token) and len(token)>=min_length,
                            words));
                            #tokens));
        return filtered_tokens
    except RuntimeError as e:
        print("Runtime Error in tokenize: {} : {}".format(e.errno, e.strerror))   


In [ ]:
def pre_process_jsonl(data_src_path, output_filename):
    heads = []
    descs = []
    keywords = None
    with open(data_src_path, 'rb') as f:
        for item in jl.reader(f):
            tit = " ".join(tokenize(item['title']))
            bod = " ".join(tokenize(item['content']))
            heads.append(tit)
            descs.append(bod)
            
    return (heads, descs, keywords)

In [ ]:
heads, desc, keywords = pre_process_jsonl(data_src_path, output_filename)

## Dump to file via pickle

In [11]:
def save_via_pickle(data, output_filename):
    with open(output_filename, 'wb') as out_f:
            pickle.dump(data, out_f)

In [12]:
save_via_pickle((heads, desc, keywords), output_filename)

OSError: [Errno 22] Invalid argument

In [1]:
heads[:20]

NameError: name 'heads' is not defined

NameError: name 'python' is not defined